
#                基于前程无忧数据的互联网就业分析


### 作者：木烨

## 需求分析


##### 问题背景
互联网行业作为目前公认的高薪岗位，互联网就业也有着很大的市场需求，每年无数毕业生进入互联网行业就业
##### 问题阐述
哪些城市对互联网职业薪酬需求更大？提供的薪酬更加诱人？

掌握哪一种编程语言在就业时更具优势，可以获得更高的薪酬？

学历、工作经验对于就业薪酬又有着怎样的影响？


从这些现实问题入手，分析互联网就业的现状。

数据集：从51job.com抓取数据进行分析统计。

主要使用：scrapy，正则表达式，SVM与随机森林预测，Kmeans聚类，BIRCH聚类，词云图，可视化方法



# 数据集

从51job.com抓取数据进行分析统计。

##### 主要使用：

scrapy，正则表达式，SVM与随机森林预测，Kmeans聚类，BIRCH聚类，词云图，可视化方法



## 一. 数据获取

利用爬虫从51job.com（前程无忧）网站搜集相关信息。
(网址链接为：https://search.51job.com/)

先后尝试爬取了拉勾网、BOSS网与前程无忧网这三个国内较大的招聘网站。

经过多次碰壁发现，拉勾网与BOSS或因为所需信息位于js中，或因为反爬虫机制过强导致爬虫易使得IP被封，而前程无忧网反爬虫机制远弱于BOSS与拉勾网，相对而言数据收集的难度，但因其可爬取数据并不整齐，因此数据清洗需要较多的工作。

In [1]:
#数据获取代码（爬虫）
import numpy as np
import pandas as pd
import pylab as pl
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
%matplotlib inline
from pylab import *
mpl.rcParams['font.sans-serif'] = ['SimHei']
import collections # 词频统计库
import jieba # 结巴分词
import wordcloud # 词云展示库
from PIL import Image # 图像处理库
import networkx as nx
import scipy as sp
import re




爬虫具体步骤如下：

1.首先对https://search.51job.com/ 爬取得到spider1.csv，由于此网站仅含有少量信息，其余信息隐藏在每个职位对应的招聘小网页。所以先从这一网站爬取具体职位对应的有详细介绍的网页网址。

2.对与小网页进行爬取，得到spider2.csv，网站示例如：https://jobs.51job.com/beijing/119317470.html?s=01&t=0 。(用时4h左右)

3.将两个csv文件进行合并得出data.csv数据集。

注爬虫代码的执行过程在另一notebook，由于那一notebook过大，容易造成jupyter崩溃，故分开安放

In [ ]:
##第一次爬取
class MySpider(scrapy.Spider): 
    ROBOTSTXT_OBEY = False
    name = "spider"
    def __init__(self):
        self.urls = []
        for i in range(1,1602):  ##北京，上海，广州，深圳，杭州
            self.urls.append(('https://search.51job.com/list/020000%252C010000%252C030200%252C040000%252C080200,000000,0100,00,9,99,%2520,2,')+str(i)+('.html'))
        for i in range(1,430):  ##西安，武汉，重庆，哈尔滨，成都
            self.urls.append(('https://search.51job.com/list/200200%252C090200%252C180200%252C060000%252C220200,000000,0100,00,9,99,%2520,2,')+str(i)+('.html'))
    def start_requests(self):      
        for url in self.urls:
            yield scrapy.Request(url=url, callback=self.parse)    
    def parse(self, response):        
        quotes = response.css('#resultList .el')
        job=[]         ##职位名称
        jobweb=[]      ##具体详细介绍的网站，之后以此为依据进行二次爬取
        companyweb=[]  ##公司对应的网站
        company=[]     ##公司名
        place=[]       ##招聘地区
        salary=[]      ##薪水
        time=[]        ##发布时间
        for quote in quotes:
            job.append(quote.css('.t1 a::attr(title)').extract_first())
            jobweb.append(quote.css('.t1 a::attr(href)').extract_first())
            company.append(quote.css('.t2 a::text').extract_first())
            companyweb.append(quote.css('.t2 a::attr(href)').extract_first())
            place.append(quote.css('.t3::text').extract_first())
            salary.append(quote.css('.t4::text').extract_first())
            time.append(quote.css('.t5::text').extract_first())
            item = {'job':job,'jobweb':jobweb,'company':company,'companyweb':companyweb,'place':place,'salary':salary,'time':time}
            df = pd.DataFrame.from_dict(data=dict(item))  
        df.dropna(subset=["companyweb"],axis=0,inplace=True) 
        df.to_csv('spider1.csv',index=0,encoding='utf-8-sig',mode='a',header=False)
        print("over: " + response.url)
from scrapy.crawler import CrawlerProcess
process = CrawlerProcess({ 
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.87 Safari/537.36'
})
process.crawl(MySpider)
process.start()

In [ ]:
##第二次爬取，由于要爬取的网页过多，爬取时间较长
class MySpider(scrapy.Spider): 
    ROBOTSTXT_OBEY = False
    name = "spider"
    def __init__(self): 
        czh=pd.read_csv('spider1.csv')   ##从爬取的第一个csv中获得具体职位介绍网站，挨个进行爬取
        web=czh.iloc[:,1]
        web=np.array(web)
        self.urls=web.tolist()
    def start_requests(self):      
        for url in self.urls:
            yield scrapy.Request(url=url, callback=self.parse)    
    def parse(self, response):        
        quotes = response.css('div.cn')         
        texts=response.css('div.mt10')
        texts2=response.css('div.bmsg.job_msg.inbox')
        texts3=response.css('div.tBorderTop_box')
        jobweb=[]        ##具体职位介绍网址，之后以此为依据，将两个spider.csv进行合并
        education=[]   
           ##职位对学历、工作经验、招聘人数等的要求，由于网站本身这些数据在一个文本框中，只能先行爬取，之后预处理
        jtap=[]         ##爬取职位福利
        duty=[]         ##爬取职责information等
        web=[]          
        worktype=[]
        for quote in quotes:
            jtap.append(quote.css('.sp4::text').extract())
            education.append(quote.css('p::attr(title)').extract_first())#.split(str='\xa0\xa0|\xa0\xa0'))
        for text in texts:
            web.append(response.url)
            worktype.append(text.css('a::text').extract())
        for text2 in texts2:
            duty.append(text2.css('p::text').extract())
        item = {'education':education,'jtap':jtap,'worktype':worktype,'duty':duty,'web':web}#'experience':experience,'number':number,'jtap':jtap,'information':information,'duty':duty,'worktype':worktype}
        df = pd.DataFrame.from_dict(data=dict(item))  
        df.to_csv('spider2.csv',index=0,encoding='utf-8-sig',mode='a',header=False)
        print("over: " + response.url)
from scrapy.crawler import CrawlerProcess
process = CrawlerProcess({ 
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.87 Safari/537.36'
})
process.crawl(MySpider)
process.start()


## 二.数据清洗





#### 需注明脏数据的判断依据以及要如何清洗
   1.原始数据存储在两个csv文件，首先将他们以爬取的网站为公共列进行合并。

   2.由于原本网站的限制，工作经验、学历要求、招聘人数全部位于jtap一列，用“|”隔开，用正则表达式提取关键词

   3.将不规整的原始薪酬数据转化统一格式，并提取数字，将万/年、元/天、千/月统一转化为万/月

   4.从job(职位名称)、key(职位关键词)、duty(对职位需求的描述)匹配所需的编程语言，并存储在不同的列中

   5.删去多余的web、companyweb等列,进行一些格式调整

In [2]:
#数据清洗代码
test1=pd.read_csv('spider1.csv', header=None, names = ['job','web','company','companyweb','place','salary','time'] )
test2=pd.read_csv('spider2.csv', header=None, names = ['jtap','text','key','duty','web'] )
data=pd.merge(test1,test2)

显示合并之后的dataframe

In [4]:
data.drop_duplicates(subset=None, keep='first', inplace=True)
data.reset_index(drop=True)
data.to_csv('data.csv',index=1,encoding='utf-8-sig',mode='a',header='True')
data

,job,web,company,companyweb,place,salary,time,jtap,text,key,duty
0,自动化测试开发工程师,https://jobs.51job.com/hangzhou-bjq/119316672....,滨江兄弟信息技术（杭州）有限公司...,https://jobs.51job.com/all/co2475040.html,杭州-滨江区,3-4.5千/月,12-18,杭州-滨江区 | 无工作经验 | 本科 | 招1人 | 12-18发布,[],"['软件工程师', 'c#', 'rpa', 'pathon', 'vba']","['公司内部业务流程自动化的开发', '负责RPA流程的开发、调试', '招聘要求：', '..."
4,中级安卓App软件工程师,https://jobs.51job.com/shenzhen-baq/119316565....,深圳软牛科技有限公司,https://jobs.51job.com/all/co3506969.html,深圳-宝安区,0.9-1.5万/月,12-18,深圳-宝安区 | 3-4年经验 | 本科 | 招2人 | 12-18发布 ...,"['五险一金', '员工旅游', '股票期权', '弹性工作', '专业培训', '下午茶活...","['软件工程师', 'Android', '客户端', 'java', 'App']","['1、负责公司项目Android客户端开发与测试工作；', '2、完成android软件系..."
12,Unity客户端主程,https://jobs.51job.com/shenzhen-nsq/119114130....,深圳市大家族网络有限公司,https://jobs.51job.com/all/co3333597.html,深圳-南山区,1.8-2.4万/月,12-18,深圳-南山区 | 3-4年经验 | 大专 | 招1人 | 12-18发布,"['五险一金', '补充医疗保险', '员工旅游', '年终奖金', '绩效奖金', '餐饮...","['多媒体/游戏开发工程师', '软件工程师', 'unity3D', '游戏开发', '前...","['（1）动作RPG手游前端开发', '（2）指导和安排前端小组的工作', '任职要求：',..."
20,嵌入式单片机软件工程师,https://jobs.51job.com/shenzhen-ftq/111802749....,凡方数码技术有限公司,https://jobs.51job.com/all/co414697.html,深圳-福田区,1-2万/月,12-18,深圳-福田区 | 2年经验 | 本科 | 招1人 | 12-18发布 | ...,"['五险一金', '员工旅游', '周末双休', '餐饮补贴', '绩效奖金']","['嵌入式软件开发(Linux/单片机/PLC/DSP…)', '软件工程师', '单片机'...","['任职资格：', '1.\xa0计算机\xa0软件工程相关大专以上学历，25岁以上；', ..."
28,PHP高级开发工程师,https://jobs.51job.com/guangzhou-thq/119298974...,广东阅云科技有限公司,https://jobs.51job.com/all/co4262775.html,广州-天河区,1-1.5万/月,12-18,广州-天河区 | 3-4年经验 | 大专 | 招2人 | 12-18发布,"['全勤奖', '绩效奖金', '节日福利', '弹性工作', '交通补贴', '年终奖金'...","['高级软件工程师', 'php,laravel']","['1、用PHP语言开发高效能数据系统，后台开发和维护；', '2、熟悉掌握laravel框..."
36,SH-230 资深软件开发工程师（5G方向）,https://jobs.51job.com/shanghai/119171019.html...,上海剑桥科技股份有限公司,https://jobs.51job.com/all/co1870293.html,上海,1.8-3.5万/月,12-18,上海 | 5-7年经验 | 本科 | 招2人 | 12-18发布 | 通...,"['五险一金', '补充医疗保险', '免费班车', '餐饮补贴', '交通补贴', '通讯...","['高级软件工程师', '软件工程师']",[' 情况介绍 ']
44,软件工程师,https://jobs.51job.com/shanghai-fxq/115462110....,上海庆易自动化设备有限公司,https://jobs.51job.com/all/co5445791.html,上海-奉贤区,0.8-1万/月,12-18,上海-奉贤区 | 2年经验 | 本科 | 招1人 | 12-18发布 | ...,[],"['软件工程师', '软件工程师']",['根据机器人等智能设备研发项目要求，承担控制系统的软件设计和开发工作，实现满足设备要求的控...
52,E采系统运维工程师,https://jobs.51job.com/beijing-hdq/119316522.h...,北京福瑞兰斯咨询有限公司,https://jobs.51job.com/all/co4532741.html,北京-海淀区,1-1.5万/月,12-18,北京-海淀区 | 无工作经验 | 招3人 | 12-18发布,['五险一金'],"['系统工程师', 'E采系统']","['负责大客户E采系统的日常运维工作', '听从IT 指导，处理系统日常问题，包括ticke..."
60,苹果平台大数据工程师,https://jobs.51job.com/beijing-hdq/119316446.h...,北京新宇智慧信息技术有限公司,https://jobs.51job.com/all/co5341535.html,北京-海淀区,1.2-2万/月,12-18,北京-海淀区 | 2年经验 | 本科 | 招若干人 | 12-18发布,"['周末双休', '节日福利', '五险一金', '员工旅游']",['软件工程师'],['在\xa0Apple\xa0北京的测试中心进行测试相关的工作（自动化测试，涉及自动化测试...
68,iOS开发工程师,https://jobs.51job.com/shenzhen-nsq/119316423....,深圳市慧创恒通科技有限公司,https://jobs.51job.com/all/co5443471.html,深圳-南山区,1-1.2万/月,12-18,深圳-南山区 | 2年经验 | 大专 | 招1人 | 12-18发布,"['五险一金', '员工旅游', '弹性工作', '年终奖金', '绩效奖金']","['软件工程师', '开发工程师']","['岗位职责：', '1.协同团队高质量完成IOS端的App开发并编写相应的技术文档；', ..."


利用正则表达式匹配，由于10w组数据过多，正则表达式效率过低，所以下一代码运行了三个小时左右。

主要用正则完成了招聘人数、经验、薪酬数据的获取，此外还进行了学历和地名的匹配与统一化

In [ ]:
# 利用正则表达式匹配招聘人数 招聘学历要求
data['education']="NA"
data['num']="NA"
data['exp']='NA'
data['money']='NA'
data['city']='NA'
column=data.shape[0]
citylist=['北京','上海','深圳','广州','杭州','西安','哈尔滨','武汉','成都','重庆']
educationlist=['博士','硕士','本科','大专','高中','中技','中专','初中及以下']

for i in range(0,data.shape[0]):
    progress = int(i*100/column)
    progerss_str = ('#'*progress)+"({}%/100%)".format(progress)
    print("\r完成进度:{:<120}".format(progerss_str),end="")      ##输出进度
    
    data.iloc[i,12]=re.search('招(.*?)人',data.iloc[i,7]).group(1)  ##匹配招聘人数
    
    ##匹配学历要求
    for m in range(0,8):
        if educationlist[m] in data.iloc[i,7]:
            data.iloc[i,11]=educationlist[m]
  
    if "无工作经验" in data.iloc[i,7]:          ##匹配工作经验要求
        data.iloc[i,13]="无"
    else:
        try:    ##匹配格式 3-4年经验
            data.iloc[i,13]=(int(re.match('.*(\d)-(\d).*经验',data.iloc[i,7]).group(1))+int(re.match('.*(\d)-(\d).*经验',data.iloc[i,7]).group(2)))/2
        except:   ##匹配格式  4年经验
            data.iloc[i,13]=int(re.match('.*(\d).*经验',data.iloc[i,7]).group(1))
            
    ##利用正则表达式匹配薪资水平，并将各种单位统一转换为 万/月，方便后续计算
    try:  ##按月结算
        if "月" in data.iloc[i,5]:
            try:  ##以万为单位
                data.iloc[i,14]=(float(re.search('(\d+(\.\d+)?)-(\d+(\.\d+)?)万/月',data.iloc[i,5]).group(1))+float(re.search('(\d+(\.\d+)?)-(\d+(\.\d+)?)万/月',data.iloc[i,5]).group(3)))/2
            except:   ##以千为单位
                data.iloc[i,14]=(float(re.search('(\d+(\.\d+)?)-(\d+(\.\d+)?)千/月',data.iloc[i,5]).group(1))+float(re.search('(\d+(\.\d+)?)-(\d+(\.\d+)?)千/月',data.iloc[i,5]).group(3)))/20  
        ##按年结算
        if "年" in data.iloc[i,5]:
            try:
                data.iloc[i,14]=(float(re.search('(\d+(\.\d+)?)-(\d+(\.\d+)?)万/年',data.iloc[i,5]).group(1))+float(re.search('(\d+(\.\d+)?)-(\d+(\.\d+)?)万/年',data.iloc[i,5]).group(3)))/24
            except:
                data.iloc[i,14]=(float(re.search('(\d+(\.\d+)?)-(\d+(\.\d+)?)千/年',data.iloc[i,5]).group(1))+float(re.search('(\d+(\.\d+)?)-(\d+(\.\d+)?)千/年',data.iloc[i,5]).group(3)))/240
        ##按天结算
        if "天" in data.iloc[i,5]:
            data.iloc[i,14]=float(re.match('(\d+(\.\d+)?)元/天',data.iloc[i,5]).group(1))*30/10000
    except:  
        data.iloc[i,14]='NA'   
        
    ##将地名具体化
    for k in range(0,10):
        if citylist[k] in data.iloc[i,4]:
            data.iloc[i,15]=citylist[k]



显示匹配之后的结果，此时的数据共100507组

In [16]:
data

,job,web,company,companyweb,place,salary,time,jtap,text,key,duty,education,num,exp,money,city
0,自动化测试开发工程师,https://jobs.51job.com/hangzhou-bjq/119316672....,滨江兄弟信息技术（杭州）有限公司...,https://jobs.51job.com/all/co2475040.html,杭州,3-4.5千/月,12-18,杭州-滨江区 | 无工作经验 | 本科 | 招1人 | 12-18发布,[],"['软件工程师', 'c#', 'rpa', 'pathon', 'vba']","['公司内部业务流程自动化的开发', '负责RPA流程的开发、调试', '招聘要求：', '...",本科,1,无,0.375,杭州
4,中级安卓App软件工程师,https://jobs.51job.com/shenzhen-baq/119316565....,深圳软牛科技有限公司,https://jobs.51job.com/all/co3506969.html,深圳,0.9-1.5万/月,12-18,深圳-宝安区 | 3-4年经验 | 本科 | 招2人 | 12-18发布 ...,"['五险一金', '员工旅游', '股票期权', '弹性工作', '专业培训', '下午茶活...","['软件工程师', 'Android', '客户端', 'java', 'App']","['1、负责公司项目Android客户端开发与测试工作；', '2、完成android软件系...",本科,2,3.5,1.2,深圳
12,Unity客户端主程,https://jobs.51job.com/shenzhen-nsq/119114130....,深圳市大家族网络有限公司,https://jobs.51job.com/all/co3333597.html,深圳,1.8-2.4万/月,12-18,深圳-南山区 | 3-4年经验 | 大专 | 招1人 | 12-18发布,"['五险一金', '补充医疗保险', '员工旅游', '年终奖金', '绩效奖金', '餐饮...","['多媒体/游戏开发工程师', '软件工程师', 'unity3D', '游戏开发', '前...","['（1）动作RPG手游前端开发', '（2）指导和安排前端小组的工作', '任职要求：',...",大专,1,3.5,2.1,深圳
20,嵌入式单片机软件工程师,https://jobs.51job.com/shenzhen-ftq/111802749....,凡方数码技术有限公司,https://jobs.51job.com/all/co414697.html,深圳,1-2万/月,12-18,深圳-福田区 | 2年经验 | 本科 | 招1人 | 12-18发布 | ...,"['五险一金', '员工旅游', '周末双休', '餐饮补贴', '绩效奖金']","['嵌入式软件开发(Linux/单片机/PLC/DSP…)', '软件工程师', '单片机'...","['任职资格：', '1.\xa0计算机\xa0软件工程相关大专以上学历，25岁以上；', ...",本科,1,2,1.5,深圳
28,PHP高级开发工程师,https://jobs.51job.com/guangzhou-thq/119298974...,广东阅云科技有限公司,https://jobs.51job.com/all/co4262775.html,广州,1-1.5万/月,12-18,广州-天河区 | 3-4年经验 | 大专 | 招2人 | 12-18发布,"['全勤奖', '绩效奖金', '节日福利', '弹性工作', '交通补贴', '年终奖金'...","['高级软件工程师', 'php,laravel']","['1、用PHP语言开发高效能数据系统，后台开发和维护；', '2、熟悉掌握laravel框...",大专,2,3.5,1.25,广州
36,SH-230 资深软件开发工程师（5G方向）,https://jobs.51job.com/shanghai/119171019.html...,上海剑桥科技股份有限公司,https://jobs.51job.com/all/co1870293.html,上海,1.8-3.5万/月,12-18,上海 | 5-7年经验 | 本科 | 招2人 | 12-18发布 | 通...,"['五险一金', '补充医疗保险', '免费班车', '餐饮补贴', '交通补贴', '通讯...","['高级软件工程师', '软件工程师']",[' 情况介绍 '],本科,2,6,2.65,上海
44,软件工程师,https://jobs.51job.com/shanghai-fxq/115462110....,上海庆易自动化设备有限公司,https://jobs.51job.com/all/co5445791.html,上海,0.8-1万/月,12-18,上海-奉贤区 | 2年经验 | 本科 | 招1人 | 12-18发布 | ...,[],"['软件工程师', '软件工程师']",['根据机器人等智能设备研发项目要求，承担控制系统的软件设计和开发工作，实现满足设备要求的控...,本科,1,2,0.9,上海
52,E采系统运维工程师,https://jobs.51job.com/beijing-hdq/119316522.h...,北京福瑞兰斯咨询有限公司,https://jobs.51job.com/all/co4532741.html,北京,1-1.5万/月,12-18,北京-海淀区 | 无工作经验 | 招3人 | 12-18发布,['五险一金'],"['系统工程师', 'E采系统']","['负责大客户E采系统的日常运维工作', '听从IT 指导，处理系统日常问题，包括ticke...",NA,3,无,1.25,北京
60,苹果平台大数据工程师,https://jobs.51job.com/beijing-hdq/119316446.h...,北京新宇智慧信息技术有限公司,https://jobs.51job.com/all/co5341535.html,北京,1.2-2万/月,12-18,北京-海淀区 | 2年经验 | 本科 | 招若干人 | 12-18发布,"['周末双休', '节日福利', '五险一金', '员工旅游']",['软件工程师'],['在\xa0Apple\xa0北京的测试中心进行测试相关的工作（自动化测试，涉及自动化测试...,本科,若干,2,1.6,北京
68,iOS开发工程师,https://jobs.51job.com/shenzhen-nsq/119316423....,深圳市慧创恒通科技有限公司,https://jobs.51job.com/all/co5443471.html,深圳,1-1.2万/月,12-18,深圳-南山区 | 2年经验 | 大专 | 招1人 | 12-18发布,"['五险一金', '员工旅游', '弹性工作', '年终奖金', '绩效奖金']","['软件工程师', '开发工程师']","['岗位职责：', '1.协同团队高质量完成IOS端的App开发并编写相应的技术文档；', ...",大专,1,2,1.1,深圳


删去一些不必要的列，并将匹配不到薪酬、教育、城市的列删去，将现有data存储到data1.csv,后续处理时可以直接读取data1.csv而不用再次运行耗时过长的模块

In [6]:
data=data[data['money']<1000]               
##在现有数据集中有4组月薪大于1000，排查后发现可能是招聘公司书写要求时不严谨，由于对后续分析干扰过大先行删去
data = data[pd.notnull(data['money'])]
data = data[pd.notnull(data['education'])]
data = data[pd.notnull(data['city'])]

In [7]:
#删去不必要的列，保存进度
data = data.drop('web', axis=1)
data=data.drop('companyweb',axis=1)
data = data.drop('time', axis=1)
data=data.drop('jtap',axis=1)
data=data.drop('salary',axis=1)
data=data.drop('place',axis=1)
data1.to_csv('data1.csv',index=False,encoding='utf-8-sig',mode='a')

In [5]:
##可直接data1.csv进行后续操作
data=pd.read_csv('data1.csv')
data.reset_index(drop=True)

,Unnamed: 0,job,web,company,companyweb,place,salary,time,jtap,text,key,duty,education,num,exp,money,city
0,0,自动化测试开发工程师,https://jobs.51job.com/hangzhou-bjq/119316672....,滨江兄弟信息技术（杭州）有限公司...,https://jobs.51job.com/all/co2475040.html,杭州,3-4.5千/月,12月18日,杭州-滨江区 | 无工作经验 | 本科 | 招1人 | 12-18发布,[],"['软件工程师', 'c#', 'rpa', 'pathon', 'vba']","['公司内部业务流程自动化的开发', '负责RPA流程的开发、调试', '招聘要求：', '...",本科,1,无,0.375000,杭州
1,1,中级安卓App软件工程师,https://jobs.51job.com/shenzhen-baq/119316565....,深圳软牛科技有限公司,https://jobs.51job.com/all/co3506969.html,深圳,0.9-1.5万/月,12月18日,深圳-宝安区 | 3-4年经验 | 本科 | 招2人 | 12-18发布 ...,"['五险一金', '员工旅游', '股票期权', '弹性工作', '专业培训', '下午茶活...","['软件工程师', 'Android', '客户端', 'java', 'App']","['1、负责公司项目Android客户端开发与测试工作；', '2、完成android软件系...",本科,2,3.5,1.200000,深圳
2,2,Unity客户端主程,https://jobs.51job.com/shenzhen-nsq/119114130....,深圳市大家族网络有限公司,https://jobs.51job.com/all/co3333597.html,深圳,1.8-2.4万/月,12月18日,深圳-南山区 | 3-4年经验 | 大专 | 招1人 | 12-18发布,"['五险一金', '补充医疗保险', '员工旅游', '年终奖金', '绩效奖金', '餐饮...","['多媒体/游戏开发工程师', '软件工程师', 'unity3D', '游戏开发', '前...","['（1）动作RPG手游前端开发', '（2）指导和安排前端小组的工作', '任职要求：',...",大专,1,3.5,2.100000,深圳
3,3,嵌入式单片机软件工程师,https://jobs.51job.com/shenzhen-ftq/111802749....,凡方数码技术有限公司,https://jobs.51job.com/all/co414697.html,深圳,1-2万/月,12月18日,深圳-福田区 | 2年经验 | 本科 | 招1人 | 12-18发布 | ...,"['五险一金', '员工旅游', '周末双休', '餐饮补贴', '绩效奖金']","['嵌入式软件开发(Linux/单片机/PLC/DSP…)', '软件工程师', '单片机'...","['任职资格：', '1.\xa0计算机\xa0软件工程相关大专以上学历，25岁以上；', ...",本科,1,2,1.500000,深圳
4,4,PHP高级开发工程师,https://jobs.51job.com/guangzhou-thq/119298974...,广东阅云科技有限公司,https://jobs.51job.com/all/co4262775.html,广州,1-1.5万/月,12月18日,广州-天河区 | 3-4年经验 | 大专 | 招2人 | 12-18发布,"['全勤奖', '绩效奖金', '节日福利', '弹性工作', '交通补贴', '年终奖金'...","['高级软件工程师', 'php,laravel']","['1、用PHP语言开发高效能数据系统，后台开发和维护；', '2、熟悉掌握laravel框...",大专,2,3.5,1.250000,广州
5,5,SH-230 资深软件开发工程师（5G方向）,https://jobs.51job.com/shanghai/119171019.html...,上海剑桥科技股份有限公司,https://jobs.51job.com/all/co1870293.html,上海,1.8-3.5万/月,12月18日,上海 | 5-7年经验 | 本科 | 招2人 | 12-18发布 | 通...,"['五险一金', '补充医疗保险', '免费班车', '餐饮补贴', '交通补贴', '通讯...","['高级软件工程师', '软件工程师']",[' 情况介绍 '],本科,2,6,2.650000,上海
6,6,软件工程师,https://jobs.51job.com/shanghai-fxq/115462110....,上海庆易自动化设备有限公司,https://jobs.51job.com/all/co5445791.html,上海,0.8-1万/月,12月18日,上海-奉贤区 | 2年经验 | 本科 | 招1人 | 12-18发布 | ...,[],"['软件工程师', '软件工程师']",['根据机器人等智能设备研发项目要求，承担控制系统的软件设计和开发工作，实现满足设备要求的控...,本科,1,2,0.900000,上海
7,7,E采系统运维工程师,https://jobs.51job.com/beijing-hdq/119316522.h...,北京福瑞兰斯咨询有限公司,https://jobs.51job.com/all/co4532741.html,北京,1-1.5万/月,12月18日,北京-海淀区 | 无工作经验 | 招3人 | 12-18发布,['五险一金'],"['系统工程师', 'E采系统']","['负责大客户E采系统的日常运维工作', '听从IT 指导，处理系统日常问题，包括ticke...",NaN,3,无,1.250000,北京
8,8,苹果平台大数据工程师,https://jobs.51job.com/beijing-hdq/119316446.h...,北京新宇智慧信息技术有限公司,https://jobs.51job.com/all/co5341535.html,北京,1.2-2万/月,12月18日,北京-海淀区 | 2年经验 | 本科 | 招若干人 | 12-18发布,"['周末双休', '节日福利', '五险一金', '员工旅游']",['软件工程师'],['在\xa0Apple\xa0北京的测试中心进行测试相关的工作（自动化测试，涉及自动化测试...,本科,若干,2,1.600000,北京
9,9,iOS开发工程师,https://jobs.51job.com/shenzhen-nsq/119316423....,深圳市慧创恒通科技有限公司,https://jobs.51job.com/all/co5443471.html,深圳,1-1.2万/月,12月18日,深圳-南山区 | 2年经验 | 大专 | 招1人 | 12-18发布,"['五险一金', '员工旅游', '弹性工作', '年终奖金', '绩效奖金']","['软件工程师', '开发工程师']","['岗位职责：', '1.协同团队高质量完成IOS端的App开发并编写相应的技术文档；', ...",大专,1,2,1.100000,深圳


In [6]:
##此时共有7.8w数据
data

,Unnamed: 0,job,web,company,companyweb,place,salary,time,jtap,text,key,duty,education,num,exp,money,city
0,0,自动化测试开发工程师,https://jobs.51job.com/hangzhou-bjq/119316672....,滨江兄弟信息技术（杭州）有限公司...,https://jobs.51job.com/all/co2475040.html,杭州,3-4.5千/月,12月18日,杭州-滨江区 | 无工作经验 | 本科 | 招1人 | 12-18发布,[],"['软件工程师', 'c#', 'rpa', 'pathon', 'vba']","['公司内部业务流程自动化的开发', '负责RPA流程的开发、调试', '招聘要求：', '...",本科,1,无,0.375000,杭州
1,1,中级安卓App软件工程师,https://jobs.51job.com/shenzhen-baq/119316565....,深圳软牛科技有限公司,https://jobs.51job.com/all/co3506969.html,深圳,0.9-1.5万/月,12月18日,深圳-宝安区 | 3-4年经验 | 本科 | 招2人 | 12-18发布 ...,"['五险一金', '员工旅游', '股票期权', '弹性工作', '专业培训', '下午茶活...","['软件工程师', 'Android', '客户端', 'java', 'App']","['1、负责公司项目Android客户端开发与测试工作；', '2、完成android软件系...",本科,2,3.5,1.200000,深圳
2,2,Unity客户端主程,https://jobs.51job.com/shenzhen-nsq/119114130....,深圳市大家族网络有限公司,https://jobs.51job.com/all/co3333597.html,深圳,1.8-2.4万/月,12月18日,深圳-南山区 | 3-4年经验 | 大专 | 招1人 | 12-18发布,"['五险一金', '补充医疗保险', '员工旅游', '年终奖金', '绩效奖金', '餐饮...","['多媒体/游戏开发工程师', '软件工程师', 'unity3D', '游戏开发', '前...","['（1）动作RPG手游前端开发', '（2）指导和安排前端小组的工作', '任职要求：',...",大专,1,3.5,2.100000,深圳
3,3,嵌入式单片机软件工程师,https://jobs.51job.com/shenzhen-ftq/111802749....,凡方数码技术有限公司,https://jobs.51job.com/all/co414697.html,深圳,1-2万/月,12月18日,深圳-福田区 | 2年经验 | 本科 | 招1人 | 12-18发布 | ...,"['五险一金', '员工旅游', '周末双休', '餐饮补贴', '绩效奖金']","['嵌入式软件开发(Linux/单片机/PLC/DSP…)', '软件工程师', '单片机'...","['任职资格：', '1.\xa0计算机\xa0软件工程相关大专以上学历，25岁以上；', ...",本科,1,2,1.500000,深圳
4,4,PHP高级开发工程师,https://jobs.51job.com/guangzhou-thq/119298974...,广东阅云科技有限公司,https://jobs.51job.com/all/co4262775.html,广州,1-1.5万/月,12月18日,广州-天河区 | 3-4年经验 | 大专 | 招2人 | 12-18发布,"['全勤奖', '绩效奖金', '节日福利', '弹性工作', '交通补贴', '年终奖金'...","['高级软件工程师', 'php,laravel']","['1、用PHP语言开发高效能数据系统，后台开发和维护；', '2、熟悉掌握laravel框...",大专,2,3.5,1.250000,广州
5,5,SH-230 资深软件开发工程师（5G方向）,https://jobs.51job.com/shanghai/119171019.html...,上海剑桥科技股份有限公司,https://jobs.51job.com/all/co1870293.html,上海,1.8-3.5万/月,12月18日,上海 | 5-7年经验 | 本科 | 招2人 | 12-18发布 | 通...,"['五险一金', '补充医疗保险', '免费班车', '餐饮补贴', '交通补贴', '通讯...","['高级软件工程师', '软件工程师']",[' 情况介绍 '],本科,2,6,2.650000,上海
6,6,软件工程师,https://jobs.51job.com/shanghai-fxq/115462110....,上海庆易自动化设备有限公司,https://jobs.51job.com/all/co5445791.html,上海,0.8-1万/月,12月18日,上海-奉贤区 | 2年经验 | 本科 | 招1人 | 12-18发布 | ...,[],"['软件工程师', '软件工程师']",['根据机器人等智能设备研发项目要求，承担控制系统的软件设计和开发工作，实现满足设备要求的控...,本科,1,2,0.900000,上海
7,7,E采系统运维工程师,https://jobs.51job.com/beijing-hdq/119316522.h...,北京福瑞兰斯咨询有限公司,https://jobs.51job.com/all/co4532741.html,北京,1-1.5万/月,12月18日,北京-海淀区 | 无工作经验 | 招3人 | 12-18发布,['五险一金'],"['系统工程师', 'E采系统']","['负责大客户E采系统的日常运维工作', '听从IT 指导，处理系统日常问题，包括ticke...",NaN,3,无,1.250000,北京
8,8,苹果平台大数据工程师,https://jobs.51job.com/beijing-hdq/119316446.h...,北京新宇智慧信息技术有限公司,https://jobs.51job.com/all/co5341535.html,北京,1.2-2万/月,12月18日,北京-海淀区 | 2年经验 | 本科 | 招若干人 | 12-18发布,"['周末双休', '节日福利', '五险一金', '员工旅游']",['软件工程师'],['在\xa0Apple\xa0北京的测试中心进行测试相关的工作（自动化测试，涉及自动化测试...,本科,若干,2,1.600000,北京
9,9,iOS开发工程师,https://jobs.51job.com/shenzhen-nsq/119316423....,深圳市慧创恒通科技有限公司,https://jobs.51job.com/all/co5443471.html,深圳,1-1.2万/月,12月18日,深圳-南山区 | 2年经验 | 大专 | 招1人 | 12-18发布,"['五险一金', '员工旅游', '弹性工作', '年终奖金', '绩效奖金']","['软件工程师', '开发工程师']","['岗位职责：', '1.协同团队高质量完成IOS端的App开发并编写相应的技术文档；', ...",大专,1,2,1.100000,深圳


对duty、key、job三列进行匹配，挖掘工作所需的语言

此代码运行时间1.5h左右

In [23]:
data2=data[pd.notnull(data['duty'])]
data2=data2[pd.notnull(data['key'])]
language=['c++','java','c#','python','go','php','matlab','swift','lua',
          'perl','delphi','kotlin','ruby','typescript','vba','rust','haskell','visual basic'
         ,'sql']
for i in range(0,19):
    tem=language[i]
    data2[tem]=0
for i in range(0,data2.shape[0]):
    data2.iloc[i,1]=data2.iloc[i,1].lower()
    data2.iloc[i,4]=data2.iloc[i,4].lower()
    data2.iloc[i,5]=data2.iloc[i,5].lower()
    for k in range(0,19):
        if language[k] in data2.iloc[i,5]:
            data2.iloc[i,k+11]=1
        if language[k] in data2.iloc[i,1]:
            data2.iloc[i,k+11]=1
        if language[k] in data2.iloc[i,4]:
            data2.iloc[i,k+11]=1
    progress = int(i*100/data2.shape[0])
    progerss_str = ('#'*progress)+"({}%/100%)".format(progress)
    print("\r完成进度:{:<120}".format(progerss_str),end="")      ##输出进度

完成进度:###################################################################################################(99%/100%)           

将未匹配到所需编程语言的组删去

In [44]:
data2['sum']=0
for i in range(0,data2.shape[0]):
    progress = int(i*100/data2.shape[0])
    progerss_str = ('#'*progress)+"({}%/100%)".format(progress)
    print("\r完成进度:{:<120}".format(progerss_str),end="")      ##输出进度
    sum=0
    for k in range(0,19):
        sum=sum+data2.iloc[i,k+11]
    data2.iloc[i,30]=sum

完成进度:###################################################################################################(99%/100%)           

In [51]:
data2=data2[data2['sum']>0]
data2.to_csv('data2.csv',index=False,encoding='utf-8-sig',mode='a')

及时存储，将现有数据存储到data2.csv中，data2.csv共51599组数据

In [7]:
data2=pd.read_csv('data2.csv')

In [8]:
data2

,Unnamed: 0,job,company,text,key,duty,education,num,exp,money,...,delphi,kotlin,ruby,typescript,vba,rust,haskell,visual basic,sql,sum
0,0,自动化测试开发工程师,滨江兄弟信息技术（杭州）有限公司...,[],"['软件工程师', 'c#', 'rpa', 'pathon', 'vba']","['公司内部业务流程自动化的开发', '负责rpa流程的开发、调试', '招聘要求：', '...",本科,1,无,0.375000,...,0,0,0,0,1,0,0,0,0,2
1,1,中级安卓app软件工程师,深圳软牛科技有限公司,"['五险一金', '员工旅游', '股票期权', '弹性工作', '专业培训', '下午茶活...","['软件工程师', 'android', '客户端', 'java', 'app']","['1、负责公司项目android客户端开发与测试工作；', '2、完成android软件系...",本科,2,3.5,1.200000,...,0,0,0,0,0,0,0,0,0,1
2,2,unity客户端主程,深圳市大家族网络有限公司,"['五险一金', '补充医疗保险', '员工旅游', '年终奖金', '绩效奖金', '餐饮...","['多媒体/游戏开发工程师', '软件工程师', 'unity3d', '游戏开发', '前...","['（1）动作rpg手游前端开发', '（2）指导和安排前端小组的工作', '任职要求：',...",大专,1,3.5,2.100000,...,0,0,0,0,0,0,0,0,0,1
3,3,嵌入式单片机软件工程师,凡方数码技术有限公司,"['五险一金', '员工旅游', '周末双休', '餐饮补贴', '绩效奖金']","['嵌入式软件开发(linux/单片机/plc/dsp…)', '软件工程师', '单片机'...","['任职资格：', '1.\xa0计算机\xa0软件工程相关大专以上学历，25岁以上；', ...",本科,1,2,1.500000,...,0,0,0,0,0,0,0,0,0,1
4,4,php高级开发工程师,广东阅云科技有限公司,"['全勤奖', '绩效奖金', '节日福利', '弹性工作', '交通补贴', '年终奖金'...","['高级软件工程师', 'php,laravel']","['1、用php语言开发高效能数据系统，后台开发和维护；', '2、熟悉掌握laravel框...",大专,2,3.5,1.250000,...,0,0,0,0,0,0,0,0,1,3
5,6,软件工程师,上海庆易自动化设备有限公司,[],"['软件工程师', '软件工程师']",['根据机器人等智能设备研发项目要求，承担控制系统的软件设计和开发工作，实现满足设备要求的控...,本科,1,2,0.900000,...,0,0,0,0,0,0,0,0,0,1
6,8,苹果平台大数据工程师,北京新宇智慧信息技术有限公司,"['周末双休', '节日福利', '五险一金', '员工旅游']",['软件工程师'],['在\xa0apple\xa0北京的测试中心进行测试相关的工作（自动化测试，涉及自动化测试...,本科,若干,2,1.600000,...,0,0,0,0,0,0,0,0,0,1
7,9,ios开发工程师,深圳市慧创恒通科技有限公司,"['五险一金', '员工旅游', '弹性工作', '年终奖金', '绩效奖金']","['软件工程师', '开发工程师']","['岗位职责：', '1.协同团队高质量完成ios端的app开发并编写相应的技术文档；', ...",大专,1,2,1.100000,...,0,0,0,0,0,0,0,0,0,1
8,10,sap business analyst,深圳市幻真科技有限公司,[],"['erp技术开发', 'erp实施顾问', 'sap', 'systems', 'busi...","['responsibilities:', '1. bachelor degree in ...",本科,1,8.5,3.500000,...,0,0,0,0,0,0,0,0,0,1
9,13,web前端开发工程师,上海开域信息科技有限公司,"['五险一金', '年终奖金', '弹性工作']",['软件工程师'],"['工作职责：', '1. 负责h5、微信公众号、小程序、pc端前端开发、交互实现及性能优化...",本科,1,3.5,1.650000,...,0,0,0,0,0,0,0,0,0,1


In [9]:
data2[data2['python']==1]

,Unnamed: 0,job,company,text,key,duty,education,num,exp,money,...,delphi,kotlin,ruby,typescript,vba,rust,haskell,visual basic,sql,sum
6,8,苹果平台大数据工程师,北京新宇智慧信息技术有限公司,"['周末双休', '节日福利', '五险一金', '员工旅游']",['软件工程师'],['在\xa0apple\xa0北京的测试中心进行测试相关的工作（自动化测试，涉及自动化测试...,本科,若干,2,1.600000,...,0,0,0,0,0,0,0,0,0,1
15,20,系统工程师,中国科学院计算机网络信息中心,"['五险一金', '补充医疗保险', '餐饮补贴', '定期体检', '年终奖金', '绩效...","['系统工程师', '系统工程师']","['岗位职责', '1.\xa0负责云基础设施系统运行维护工作。', '2.\xa0负责系统...",本科,3,1,1.250000,...,0,0,0,0,0,0,0,0,1,4
43,83,软件培训售前工程师,上海锐格软件有限公司,"['五险一金', '专业培训', '年终奖金', '周末双休', '带薪年假']","['高级软件工程师', '软件工程师', 'java开发', 'java工程师', 'jav...",['我公司主要给高校计算机专业开发软件实训平台和实训素材，并提供培训服务。需要有实际开发经验...,大专,若干,2,1.000000,...,0,0,0,0,0,0,0,0,0,2
45,90,实施工程师（13薪/python）,近铁国际物流（深圳）有限公司,"['年终奖金', '周末双休', '加班补贴', '七险一金', '超长带薪年假', '全额...","['erp实施顾问', '实施', '维护', 'python', 'sql', '13薪'...","['：', '1、负责wms系统基础信息的筹建、维护、优化统计；', '2、掌握wms系统配...",本科,3,1,0.800000,...,0,0,0,1,0,0,0,0,1,3
47,93,h5前端中级工程师,广州远大信息发展有限公司,"['周末双休', '包住宿', '专业培训', '绩效奖金', '餐饮补贴', '带薪年假'...","['软件工程师', 'web前端开发', '前端开发', '工程师']","['1、负责web及移动端的前端开发工作', '2、按照项目计划，在保证质量的前提下、按时完...",大专,若干,2,0.600000,...,0,0,0,0,0,0,0,0,0,3
50,97,python开发工程师,传音控股上海研发中心,"['五险一金', '交通补贴', '餐饮补贴', '通讯补贴', '免费班车', '专业培训...",['高级软件工程师'],"['负责移动互联中心的软件开发和维护工作', '负责承接新的需求，确保按时按质按量完成开发任...",本科,1,6,2.250000,...,0,0,0,0,0,0,0,0,0,1
58,111,数据架构师,北京亿信华辰软件有限责任公司,"['五险一金', '补充医疗保险', '餐饮补贴', '专业培训', '年终奖金', '定期...",['数据库工程师/管理员'],"['工作职责:', '1、参与数据仓库建设，负责数据仓库模型设计；', '2、设计并开发高质...",本科,若干,2,2.500000,...,0,0,0,0,0,0,0,0,0,2
59,112,软件工程师,广东兰贝斯科技有限公司,"['五险一金', '定期体检', '通讯补贴', '补充医疗保险', '专业培训', '员工...",['软件工程师'],"['1、负责光学字符识别（ocr）的算法研究和工程实现；', '2、通过深度学习进行模式识别...",本科,1,2,1.150000,...,0,0,0,0,0,0,0,0,0,2
74,138,java高级开发工程师,深圳市中装建设集团股份有限公司,"['五险一金', '绩效奖金', '补充医疗保险', '补充公积金']",['高级软件工程师'],"['岗位职责', '1.\xa0负责软件的程序设计与代码实现\xa0', '2.\xa0根据...",大专,2,无,1.750000,...,0,0,0,0,0,0,0,0,1,3
80,147,it系统支持岗(python/c#),深圳市联畅信息技术有限公司,"['五险一金', '员工旅游', '年终奖金', '周末双休', '岗位晋升', '节日福利...","['系统工程师', '技术支持/维护工程师']","['1、现有工具更新、修改；\xa0', '2、桌面运维工具开发；', '3、软件、硬件测试...",本科,1,2,1.250000,...,0,0,0,0,0,0,0,0,0,2


by 木烨



2019.12.24